In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import os

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 



In [4]:
size_of_image = [640, 480]

def calculate_angle_for_left_curl(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    shoulder = [left_shoulder.x, left_shoulder.y]
    elbow = [left_elbow.x, left_elbow.y]
    wrist = [left_wrist.x, left_wrist.y]

    angle = calculate_angle(shoulder, elbow, wrist)

    position_of_text = np.multiply(elbow, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_right_curl(landmarks):
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

    shoulder = [right_shoulder.x, right_shoulder.y]
    elbow = [right_elbow.x, right_elbow.y]
    wrist = [right_wrist.x, right_wrist.y]

    angle = calculate_angle(shoulder, elbow, wrist)

    position_of_text = np.multiply(elbow, size_of_image).astype(int)

    return angle, position_of_text

In [5]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

# VID_DIM = (1290, 1080)
counter = 0
cap = cv2.VideoCapture(0)
stage = None
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break
    image = cv2.flip(image, 1)
    # image = cv2.resize(image, VID_DIM)
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    try:
        landmarks = results.pose_landmarks.landmark
        left_curl_angle, left_curl_position_of_text = calculate_angle_for_left_curl(landmarks)
        
        # Visualize angle
        cv2.putText(image, 
                    str(left_curl_angle), 
                    tuple(left_curl_position_of_text), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 
                    2, 
                    cv2.LINE_AA
        )
        rigth_curl_angle, right_curl_position_of_text = calculate_angle_for_right_curl(landmarks)
        
        # Visualize angle
        cv2.putText(image, 
                    str(rigth_curl_angle), 
                    tuple(right_curl_position_of_text), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 
                    2, 
                    cv2.LINE_AA
        )
        # Curl counter logic
        if rigth_curl_angle > 160:
            stage = "down"
        if rigth_curl_angle < 30 and stage =='down':
            stage="up"
            counter +=1
            print(counter)
                    
    except:
        pass
    
    # Render curl counter
    # Setup status box
    cv2.rectangle(image, (0,0), (325,73), (245,117,16), -1)
    
    # Rep data
    cv2.putText(image, 'REPS', (15,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter), 
                (10,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    
    # Stage data
    cv2.putText(image, 'STAGE', (115,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, stage, 
                (100,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# Folder for Data collection

In [66]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('ExerciseData') 

# Actions that we try to detect
exercises = np.array(['squats', "not_squats"])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [67]:
for exercise in exercises: 
    exercise_dir = os.path.join(DATA_PATH, exercise)
    exercise_files = os.listdir(exercise_dir)

    # Check if the directory is not empty
    if exercise_files:
        # Find the maximum directory number inside the current exercise folder
        dirmax = np.max(np.array(exercise_files).astype(int))
    else:
        # If the directory is empty, start sequence numbering from 1
        dirmax = 0
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, exercise, str(dirmax+sequence)))
        except:
            pass

# Collect Data using webcam

In [68]:
def mediapipe_detection(image, model):
    image = cv2.flip(image, 1)
    # image = cv2.resize(image, VID_DIM)
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image, results

In [69]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

In [70]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    # 21 landmarks per hand, by 3 coordinates per landmark, x, y, z
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, lh, rh])

In [72]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for exercise in exercises:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(exercise, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(exercise, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints, saving each data as a numpy array
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, exercise, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# Preprocess data

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.utils import to_categorical

In [74]:
label_map = {"squats": 1, "not_squats": 0}
sequences, labels = [], []
for exercise in exercises:
    for sequence in range(start_folder, start_folder+no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, exercise, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[exercise])

In [75]:
np.array(sequences).shape

(60, 30, 258)

In [76]:
np.array(labels).shape

(60,)

In [77]:
x = np.array(sequences)
x.shape

(60, 30, 258)

In [78]:
y = to_categorical(labels).astype(int)
y.shape

(60, 2)

In [79]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)


In [80]:
y_test.shape

(3, 2)

# Build and train LSTM NN

In [81]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [82]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [83]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(exercises.shape[0], activation='softmax'))

In [84]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])


In [85]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])


Epoch 1/2000
2/2 [==============================] - 6s 51ms/step - loss: 0.6877 - categorical_accuracy: 0.5789
Epoch 2/2000
2/2 [==============================] - 0s 46ms/step - loss: 0.8410 - categorical_accuracy: 0.4912
Epoch 3/2000
2/2 [==============================] - 0s 42ms/step - loss: 0.6343 - categorical_accuracy: 0.7895
Epoch 4/2000
2/2 [==============================] - 0s 46ms/step - loss: 0.6828 - categorical_accuracy: 0.5088
Epoch 5/2000
2/2 [==============================] - 0s 48ms/step - loss: 0.6805 - categorical_accuracy: 0.5088
Epoch 6/2000
2/2 [==============================] - 0s 38ms/step - loss: 0.6731 - categorical_accuracy: 0.5439
Epoch 7/2000
2/2 [==============================] - 0s 44ms/step - loss: 0.6575 - categorical_accuracy: 0.6667
Epoch 8/2000
2/2 [==============================] - 0s 44ms/step - loss: 0.6225 - categorical_accuracy: 0.7368
Epoch 9/2000
2/2 [==============================] - 0s 47ms/step - loss: 0.5909 - categorical_accuracy: 0.7544
E

In [86]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 2)                 66        
                                                                 
Total params: 237218 (926.63 KB)
Trainable params: 237

# make predictions

In [88]:
res = model.predict(x_test)


1/1 [==============================] - 0s 34ms/step


In [92]:
exercises[np.argmax(res[1])]


'not_squats'

In [93]:
model.save('squat.h5')

c:\Users\chengyang\Physio\my_env\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# predict

In [94]:
def prob_viz(res, exercises, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, exercises[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [103]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(exercises[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic

            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if exercises[np.argmax(res)] != sentence[-1]:
                            sentence.append(exercises[np.argmax(res)])
                    else:
                        sentence.append(exercises[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, exercises, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (30,) + inhomogeneous part.

In [96]:
np.expand_dims(x_test[0], axis=0)

array([[[ 0.54226339,  0.35443488, -0.3216539 , ...,  0.60500538,
          0.66471875, -0.00400319],
        [ 0.54516768,  0.3660062 , -0.34602892, ...,  0.60574549,
          0.65421098, -0.00390442],
        [ 0.54533255,  0.36988154, -0.32190681, ...,  0.60483843,
          0.65157747, -0.00433312],
        ...,
        [ 0.56577373,  0.33437985, -0.23432238, ...,  0.62403744,
          0.60775757, -0.00290992],
        [ 0.56876951,  0.3302609 , -0.23136024, ...,  0.62729043,
          0.59787148, -0.00122698],
        [ 0.57138187,  0.32571784, -0.22644135, ...,  0.62663823,
          0.59420663,  0.0009907 ]]])

In [98]:
model.predict(np.expand_dims(x_test[0], axis=0))[0]

1/1 [==============================] - 0s 35ms/step


array([1.1299022e-23, 1.0000000e+00], dtype=float32)